<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/case-study-5--predicting-future-friendships/01_graph_theory_and_network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##An introduction to graph theory and network analysis

Network theory is the study of connections between objects. These objects can be
anything: 

* people connected by relationships, 
* web pages connected by web links, or
* cities connected by roads. 

A collection of objects and their dispersed connections is
called either a network or a graph, depending on whom you ask. 

Engineers prefer to use the term network, while mathematicians prefer graph.

Graphs are simple abstractions that
capture the complexity of our entangled, interconnected world. 

Properties of graphs
remain surprisingly consistent across systems in society and nature.

Graph theory is a
framework for mathematically tracking these consistencies. It combines ideas from diverse branches of mathematics, including probability theory and matrix analysis.

These ideas can be used to gain useful real-world insights ranging from:

* search engine page rankings 
* social circle clustering

So some knowledge of graph theory is indispensable to doing good data science.

##Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re
import glob
import time
import numpy as np
import pandas as pd


from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

##Ranking websites by popularity